## <b><font color='darkblue'>Introduction to Conversational Context: Session, State, and Memory</font></b>

### <b><font color='darkgreen'>Why Context Matters</font></b>
([source](https://google.github.io/adk-docs/sessions/#why-context-matters)) <font size='3ptx'><b>Meaningful, multi-turn conversations require agents to understand context. Just like humans, they need to recall the conversation history: what's been said and done to maintain continuity and avoid repetition.</b> The [**Agent Development Kit**](https://google.github.io/adk-docs/) (ADK) provides structured ways to manage this context through `Session`, `State`, and `Memory`. 

### <b><font color='darkgreen'>Core Concepts</font></b>
<font size='3ptx'>Think of different instances of your conversations with the agent as distinct <b>conversation threads</b>, potentially drawing upon <b>long-term knowledge</b>.</font>

1. <b><font size='3ptx'>Session</font></b>: The Current Conversation Thread
     - Represents a single, ongoing interaction between a user and your agent system.
     - Contains the chronological sequence of messages and actions taken by the agent (referred to `Events`) during that specific interaction.
     - A `Session` can also hold temporary data (`State`) relevant only during this conversation.
2. <b><font size='3ptx'>State (session.state)</font></b>: Data Within the Current Conversation.
     - Data stored within a specific `Session`.
     - Used to manage information relevant only to the current, active conversation thread (<font color='brown'>e.g., items in a shopping cart during this chat, user preferences mentioned in this session</font>).
3. <b><font size='3ptx'>Memory</font></b>: Searchable, Cross-Session Information.
     - Represents a store of information that might span multiple past sessions or include external data sources.
     - It acts as a knowledge base the agent can search to recall information or context beyond the immediate conversation.

### <b><font color='darkgreen'>Managing Context: Services</font></b>
<b><font size='3ptx'>ADK provides services to manage these concepts:</font></b>

1. <b><font size='3ptx'>SessionService</font></b>: Manages the different conversation threads (`Session` objects)
     - <b>Handles the lifecycle</b>: creating, retrieving, updating (<font color='brown'>appending `Event`s, modifying `State`</font>), and deleting individual Sessions.
2. <b><font size='3ptx'>MemoryService</font></b>: Manages the Long-Term Knowledge Store (`Memory`)
     - Handles ingesting information (<font color='brown'>often from completed `Session`s</font>) into the long-term store.
     - Provides methods to search this stored knowledge based on queries.

<b><font size='3ptx'>Implementations</font></b>: ADK offers different implementations for both `SessionService` and `MemoryService`, allowing you to choose the storage backend that best fits your application's needs. <b>Notably, in-memory implementations are provided for both services; these are designed specifically for local testing and fast development</b>. It's important to remember that all data stored using these in-memory options (<font color='brown'>sessions, state, or long-term knowledge</font>) is lost when your application restarts. For persistence and scalability beyond local testing, ADK also offers cloud-based and database service options.

<b><font size='3ptx'>In Summary:</font></b>
- <b>Session & State:</b> Focus on the current interaction – the history and data of the single, active conversation. Managed primarily by a `SessionService`.
- <b>Memory</b>: Focuses on the past and external information – a searchable archive potentially spanning across conversations. Managed by a `MemoryService`.

### <b><font color='darkgreen'>What's Next?</font></b>
In the following sections, we'll dive deeper into each of these components:
* <b><font size='3ptx'>Session</font></b>: Understanding its structure and `Event`s.
* <b><font size='3ptx'>State</font></b>: How to effectively read, write, and manage session-specific data.
* <b><font size='3ptx'>SessionService</font></b>: Choosing the right storage backend for your sessions.
* <b><font size='3ptx'>MemoryService</font></b>: Exploring options for storing and retrieving broader context.

Understanding these concepts is fundamental to building agents that can engage in complex, stateful, and context-aware conversations.

## <b><font color='darkblue'>Session: Tracking Individual Conversations</font></b>
([source](https://google.github.io/adk-docs/sessions/session/)) <font size='3ptx'><b>Following our Introduction, let's dive into the `Session`.</b> Think back to the idea of a "conversation thread." Just like you wouldn't start every text message from scratch, agents need context regarding the ongoing interaction.</font>

`Session` is the ADK object designed specifically to track and manage these individual conversation threads.

### <b><font color='darkgreen'>The Session Object</font></b>
<font size='3ptx'><b>When a user starts interacting with your agent, the `SessionService` creates a `Session` object (<font color='blue'>google.adk.sessions.Session</font>).</b> This object acts as the container holding everything related to that one specific chat thread. Here are its key properties:</font>

* <font size='3ptx'>**Identification (`id, appName, userId`)**</font>: Unique labels for the conversation.
    - **id**: A unique identifier for this specific conversation thread, essential for retrieving it later. A `SessionService` object can handle multiple `Session`(s). This field identifies which particular session object are we referring to. For example, "test_id_modification".
    - **app_name**: Identifies which agent application this conversation belongs to. For example, "id_modifier_workflow".
    - **userId**: Links the conversation to a particular user.
* <font size='3ptx'><b>History (events)</b></font>: A chronological sequence of all interactions (<font color='brown'>`Event` objects – user messages, agent responses, tool actions</font>) that have occurred within this specific thread.
* <font size='3ptx'><b>Session State (`state`)</b></font>: A place to store temporary data relevant only to this specific, ongoing conversation. This acts as a scratchpad for the agent during the interaction. We will cover how to use and manage state in detail in the next section.
* <font size='3ptx'><b>Activity Tracking (`lastUpdateTime`)</b></font>: A timestamp indicating the last time an event occurred in this conversation thread.

#### <b>Example: Examining Session Properties</b>

In [2]:
from google.adk.sessions import InMemorySessionService, Session

# Create a simple session to examine its properties
temp_service = InMemorySessionService()
example_session = await temp_service.create_session(
    app_name="my_app",
    user_id="example_user",
    state={"initial_key": "initial_value"} # State can be initialized
)

print(f"--- Examining Session Properties ---")
print(f"ID (`id`):                {example_session.id}")
print(f"Application Name (`app_name`): {example_session.app_name}")
print(f"User ID (`user_id`):         {example_session.user_id}")
print(f"State (`state`):           {example_session.state}") # Note: Only shows initial state here
print(f"Events (`events`):         {example_session.events}") # Initially empty
print(f"Last Update (`last_update_time`): {example_session.last_update_time:.2f}")
print(f"---------------------------------")

# Clean up (optional for this example)
temp_service = await temp_service.delete_session(
    app_name=example_session.app_name,
    user_id=example_session.user_id,
    session_id=example_session.id)
print("The final status of temp_service - ", temp_service)

--- Examining Session Properties ---
ID (`id`):                25af1408-e0be-48d4-943f-3cd90be29815
Application Name (`app_name`): my_app
User ID (`user_id`):         example_user
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1753798583.77
---------------------------------
The final status of temp_service -  None


<b><font color='orange'>Note</font></b>: The `state` shown above is only the initial state. State updates happen via events, as discussed in the State section.

### <b><font color='darkgreen'>Managing `Session`s with a `SessionService`</font></b>
<font size='3ptx'><b>As seen above, you don't typically create or manage `Session` objects directly. Instead, you use a `SessionService`.</b> This service acts as the central manager responsible for the entire lifecycle of your conversation sessions.</font>

Its core responsibilities include:
* <b><font size='3ptx'>Starting New Conversations</font></b>: Creating fresh `Session` objects when a user begins an interaction.
* <b><font size='3ptx'>Resuming Existing Conversations</font></b>: Retrieving a specific `Session` (<font color='brown'>using its ID</font>) so the agent can continue where it left off.
* <b><font size='3ptx'>Saving Progress</font></b>: Appending new interactions (`Event` objects) to a session's history. This is also the mechanism through which session state gets updated (more in the State section).
* <b><font size='3ptx'>Listing Conversations</font></b>: Finding the active session threads for a particular user and application.
* <b><font size='3ptx'>Cleaning Up</font></b>: Deleting `Session` objects and their associated data when conversations are finished or no longer needed.

### <b><font color='darkgreen'>`SessionService` Implementations</font></b>
([source](https://google.github.io/adk-docs/sessions/session/#session-tracking-individual-conversations)) <b><font size='3ptx'>ADK provides different `SessionService` implementations, allowing you to choose the storage backend that best suits your needs:</font></b>

1. <b><font size='3ptx'>InMemorySessionService</font></b>
    - <b>How it works</b>: Stores all session data directly in the application's memory.
    - <b>Persistence</b>: None. All conversation data is lost if the application restarts.
    - <b>Requires</b>: Nothing extra.
    - <b>Best for</b>: Quick development, local testing, examples, and scenarios where long-term persistence isn't required.

```python
 from google.adk.sessions import InMemorySessionService
 session_service = InMemorySessionService()
```

2. <b><font size='3ptx'>VertexAiSessionService</font></b>
    - <b>How it works</b>: Uses Google Cloud's Vertex AI infrastructure via API calls for session management.
    - <b>Persistence</b>: Yes. Data is managed reliably and scalably via [Vertex AI Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/).
    - <b>Requires:</b>
        - A Google Cloud project (`pip install vertexai`)
        - A Google Cloud storage bucket that can be configured by this [step](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project#storage).
        - A Reasoning Engine resource name/ID that can setup following [this tutorial](https://google.github.io/adk-docs/deploy/agent-engine/).
        - If you do not have a Google Cloud project and you want to try the VertexAiSessionService for free, see how to [try Session and Memory for free](https://google.github.io/adk-docs/sessions/express-mode/).
    - <b>Best for</b>: Scalable production applications deployed on Google Cloud, especially when integrating with other Vertex AI features.

```python
# Requires: pip install google-adk[vertexai]
# Plus GCP setup and authentication
from google.adk.sessions import VertexAiSessionService

PROJECT_ID = "your-gcp-project-id"
LOCATION = "us-central1"
# The app_name used with this service should be the Reasoning Engine ID or name
REASONING_ENGINE_APP_NAME = "projects/your-gcp-project-id/locations/us-central1/reasoningEngines/your-engine-id"

session_service = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)
# Use REASONING_ENGINE_APP_NAME when calling service methods, e.g.:
# session_service = await session_service.create_session(app_name=REASONING_ENGINE_APP_NAME, ...)
```

3. <b><font size='3ptx'>DatabaseSessionService</font></b>
    - <b>How it works</b>: Connects to a relational database (<font color='brown'>e.g., PostgreSQL, MySQL, SQLite</font>) to store session data persistently in tables.
    - <b>Persistence</b>: Yes. Data survives application restarts.
    - <b>Requires</b>: A configured database.
    - <b>Best for</b>: Applications needing reliable, persistent storage that you manage yourself.

```python
from google.adk.sessions import DatabaseSessionService
# Example using a local SQLite file:
db_url = "sqlite:///./my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)
```

Choosing the right `SessionService` is key to defining how your agent's conversation history and temporary data are stored and persist.

### <b><font color='darkgreen'>The Session Lifecycle</font></b>
([source](https://google.github.io/adk-docs/sessions/session/#the-session-lifecycle)) 

![flow](https://google.github.io/adk-docs/assets/session_lifecycle.png)

Here’s a simplified flow of how `Session` and `SessionService` work together during a conversation turn:
1. <b><font size='3ptx'>Start or Resume</font></b>: Your application's `Runner` uses the `SessionService` to either `create_session` (<font color='brown'>for a new chat</font>) or `get_session` (<font color='brown'>to retrieve an existing one</font>).
2. <b><font size='3ptx'>Context Provided</font></b>: The `Runner` gets the appropriate `Session` object from the appropriate service method, providing the agent with access to the corresponding `Session`'s `state` and `events`.
3. <b><font size='3ptx'>Agent Processing</font></b>: The user prompts the agent with a query. The agent analyzes the query and potentially the session `state` and `events` history to determine the response.
4. <b><font size='3ptx'>Response & State Update</font></b>: The agent generates a response (<font color='brown'>and potentially flags data to be updated in the state</font>). The `Runner` packages this as an `Event`.
5. <b><font size='3ptx'>Save Interaction</font></b>: The `Runner` calls `sessionService.append_event(session, event)` with the `session` and the new `event` as the arguments. The service adds the `Event` to the history and updates the session's `state` in storage based on information within the event. The session's `last_update_time` also get updated.
6. <b><font size='3ptx'>Ready for Next</font></b>: The agent's response goes to the user. The updated `Session` is now stored by the `SessionService`, ready for the next turn (<font color='brown'>which restarts the cycle at step 1, usually with the continuation of the conversation in the current session</font>).
7. <b><font size='3ptx'>End Conversation</font></b>: When the conversation is over, your application calls `sessionService.delete_session(...)` to clean up the stored session data if it is no longer required.

This cycle highlights how the `SessionService` ensures conversational continuity by managing the history and state associated with each `Session` object.

## <b><font color='darkblue'>State: The Session's Scratchpad</font></b>
([source](https://google.github.io/adk-docs/sessions/state/#state-the-sessions-scratchpad)) <font size='3ptx'><b>Within each `Session` (our conversation thread), the `state` attribute acts like the agent's dedicated scratchpad for that specific interaction.</b> While `session.events` holds the full history, `session.state` is where the agent stores and updates dynamic details needed during the conversation.</font>

### <b><font color='darkgreen'>What is `session.state`?</font></b>
<font size='3ptx'><b>Conceptually, `session.state` is a collection (dictionary or Map) holding key-value pairs.</b>  It's designed for information the agent needs to recall or track to make the current conversation effective:</font>

* <b><font size='3ptx'>Personalize Interaction</font></b>: Remember user preferences mentioned earlier (e.g., `'user_preference_theme': 'dark'`).
* <b><font size='3ptx'>Track Task Progress</font></b>: Keep tabs on steps in a multi-turn process (e.g., `'booking_step': 'confirm_payment'`).
* <b><font size='3ptx'>Accumulate Information</font></b>: Build lists or summaries (e.g., `'shopping_cart_items': ['book', 'pen']`).
* <b><font size='3ptx'>Make Informed Decisions</font></b>: Store flags or values influencing the next response (e.g., `'user_is_authenticated': True`).

#### <b>Key Characteristics of `State`</b>

1. <b><font size='3ptx'>Structure</font></b>: Serializable Key-Value Pairs
     - Data is stored as `key: value`.
     - <b>Keys</b>: Always strings (`str`). Use clear names (e.g., `'departure_city'`, `'user:language_preference'`).
     - <b>Values</b>: Must be <b>serializable</b>. This means they can be easily saved and loaded by the `SessionService`. Stick to basic types in the specific languages (Python/ Java) like strings, numbers, booleans, and simple lists or dictionaries containing only these basic types. (See API documentation for precise details).
     - ⚠️ <b>Avoid Complex Objects: Do not store non-serializable objects</b> (<font color='brown'>custom class instances, functions, connections, etc.</font>) directly in the state. Store simple identifiers if needed, and retrieve the complex object elsewhere.

2. <b><font size='3ptx'>Mutability: It Changes</font></b>
     - The contents of the state are expected to change as the conversation evolves.
  
3. <b><font size='3ptx'>Persistence: Depends on `SessionService`</font></b>
    - Whether state survives application restarts depends on your chosen service:
        - <b>InMemorySessionService</b>: Not Persistent. State is lost on restart.
        - <b>DatabaseSessionService / VertexAiSessionService</b>: Persistent. State is saved reliably.

#### <b>Organizing State with Prefixes: Scope Matters</b>
<b><font size='3ptx'>Prefixes on state keys define their scope and persistence behavior</font></b>, especially with persistent services:
* <b><font size='3pt'>No Prefix (Session State)</font></b>:
    - <b>Scope</b>: Specific to the current session (id).
    - <b>Persistence</b>: Only persists if the `SessionService` is persistent (<font color='brown'>Database, VertexAI</font>).
    - <b>Use Cases</b>: Tracking progress within the current task (<font color='brown'>e.g., `'current_booking_step'`</font>), temporary flags for this interaction (<font color='brown'>e.g., `'needs_clarification'`</font>).
    - <b>Example</b>: `session.state['current_intent'] = 'book_flight'`
* <b><font size='3pt'>`user:` Prefix (User State)</font></b>:
    - <b>Scope</b>: Tied to the `user_id`, shared across all sessions for that user (<font color='brown'>within the same `app_name`</font>).
    - <b>Persistence</b>: Persistent with Database or VertexAI. (<font color='brown'>Stored by InMemory but lost on restart</font>).
    - <b>Use Cases</b>: User preferences (<font color='brown'>e.g., `'user:theme'`</font>), profile details (<font color='brown'>e.g., `'user:name'`</font>).
    - <b>Example</b>: `session.state['user:preferred_language'] = 'fr'`
* <b><font size='3pt'>`app:` Prefix (App State):</font></b>
    - <b>Scope</b>: Tied to the app_name, shared across all users and sessions for that application.
    - <b>Persistence</b>: Persistent with Database or VertexAI. (Stored by InMemory but lost on restart).
    - <b>Use Cases</b>: Global settings (<font color='brown'>e.g., `'app:api_endpoint'`</font>), shared templates.
    - <b>Example</b>: `session.state['app:global_discount_code'] = 'SAVE10'`
* <b><font size='3pt'>`temp:` Prefix (Temporary Session State):</font></b>
    - <b>Scope</b>: Specific to the current session processing turn.
    - <b>Persistence</b>: Never Persistent. Guaranteed to be discarded, even with persistent services.
    - <b>Use Cases</b>: Intermediate results needed only immediately, data you explicitly don't want stored.
    - <b>Example</b>: `session.state['temp:raw_api_response'] = {...}`

<font size='3ptx'><b>How the Agent Sees It</b></font>: Your agent code interacts with the combined state through the single `session.state` collection (`dict`). The `SessionService` handles fetching/merging state from the correct underlying storage based on prefixes.

### <font color='darkgreen'><b>Accessing Session State in Agent Instructions</b></font>
<font size='3ptx'><b>When working with `LlmAgent` instances, you can directly inject session state values into the agent's instruction string using a simple templating syntax.</b> This allows you to create dynamic and context-aware instructions without relying solely on natural language directives.</font> 

#### <b>Using `{key}` Templating</b>
<b>To inject a value from the session state, enclose the key of the desired state variable within curly braces: `{key}`.</b> The framework will automatically replace this placeholder with the corresponding value from session.state before passing the instruction to the LLM.

* <b>Example</b>
```python
from google.adk.agents import LlmAgent

story_generator = LlmAgent(
    name="StoryGenerator",
    model="gemini-2.0-flash",
    instruction="""Write a short story about a cat, focusing on the theme: {topic}."""
)

# Assuming session.state['topic'] is set to "friendship", the LLM 
# will receive the following instruction:
# "Write a short story about a cat, focusing on the theme: friendship."
```

#### <b>Important Considerations</b>
* <b><font size='3ptx'>Key Existence</font></b>: Ensure that the key you reference in the instruction string exists in the session.state. If the key is missing, the agent might misbehave or throw an error.
* <b><font size='3ptx'>Data Types</font></b>: The value associated with the key should be a string or a type that can be easily converted to a string.
* <b><font size='3ptx'>Escaping</font></b>: If you need to use literal curly braces in your instruction (<font color='brown'>e.g., for JSON formatting</font>), you'll need to escape them.

#### <b>Bypassing State Injection with `InstructionProvider`</b>
<font size='3ptx'><b>In some cases, you might want to use `{{` and `}}` literally in your instructions without triggering the state injection mechanism.</b></font> For example, you might be writing instructions for an agent that helps with a templating language that uses the same syntax.

To achieve this, you can provide a function to the `instruction` parameter instead of a string. This function is called an <b><font color='blue'>InstructionProvider</font></b>. When you use an <b><font color='blue'>InstructionProvider</font></b>, the ADK will not attempt to inject state, and your instruction string will be passed to the model as-is.

The <b><font color='blue'>InstructionProvider</font></b> function receives a <b><font color='blue'>ReadonlyContext</font></b> object, which you can use to access session state or other contextual information if you need to build the instruction dynamically. e.g.:

```python
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext

# This is an InstructionProvider
def my_instruction_provider(context: ReadonlyContext) -> str:
    # You can optionally use the context to build the instruction
    # For this example, we'll return a static string with literal braces.
    return "This is an instruction with {{literal_braces}} that will not be replaced."

agent = LlmAgent(
    model="gemini-2.0-flash",
    name="template_helper_agent",
    instruction=my_instruction_provider)
```

If you want to both use an <b><font color='blue'>InstructionProvider</font></b> and inject state into your instructions, you can use the `inject_session_state` utility function. e.g.:

```python
from google.adk.agents import LlmAgent
from google.adk.agents.readonly_context import ReadonlyContext
from google.adk.utils import instructions_utils

async def my_dynamic_instruction_provider(context: ReadonlyContext) -> str:
    template = "This is a {adjective} instruction with {{literal_braces}}."
    # This will inject the 'adjective' state variable but leave the literal braces.
    return await instructions_utils.inject_session_state(template, context)

agent = LlmAgent(
    model="gemini-2.0-flash",
    name="dynamic_template_helper_agent",
    instruction=my_dynamic_instruction_provider
)
```

<b><font size='3ptx'>Benefits of Direct Injection:</font></b>
* <b>Clarity</b>: Makes it explicit which parts of the instruction are dynamic and based on session state.
* <b>Reliability</b>: Avoids relying on the LLM to correctly interpret natural language instructions to access state.
* <b>Maintainability</b>: Simplifies instruction strings and reduces the risk of errors when updating state variable names.

<b><font size='3ptx'>Relation to Other State Access Methods</font></b>

This direct injection method is specific to `LlmAgent` instructions. Refer to the following section for more information on other state access methods.

### <b><font color='darkgreen'>How State is Updated: Recommended Methods</font></b>
When you need to change the session state, the correct and safest method is to directly modify the state object on the `Context` provided to your function (e.g., `callback_context.state['my_key'] = 'new_value'`). This is considered "direct state manipulation" in the right way, as the framework automatically tracks these changes.

This is critically different from directly modifying the state on a `Session` object you retrieve from the `SessionService` (e.g., `my_session.state['my_key'] = 'new_value'`). <b><font color='darkred'>You should avoid this, as it bypasses the ADK's event tracking and can lead to lost data</font></b>. The "Warning" section at the end of this page has more details on this important distinction.

<b>State should always be updated as part of adding an `Event` to the session history using `session_service.append_event()`</b>. This ensures changes are tracked, persistence works correctly, and updates are thread-safe.

#### <b>1. The Easy Way: `output_key` (for Agent Text Responses)</b>
<b>This is the simplest method for saving an agent's final text response directly into the state</b>. When defining your `LlmAgent`, specify the `output_key`:

In [3]:
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.runners import Runner
from google.genai.types import Content, Part

In [4]:
# Define agent with output_key
greeting_agent = LlmAgent(
    name="Greeter",
    model="gemini-2.0-flash", # Use a valid model
    instruction="Generate a short, friendly greeting.",
    output_key="last_greeting" # Save response to state['last_greeting']
)

In [5]:
# --- Setup Runner and Session ---
app_name, user_id, session_id = "state_app", "user1", "session1"
session_service = InMemorySessionService()
runner = Runner(
    agent=greeting_agent,
    app_name=app_name,
    session_service=session_service
)
session = await session_service.create_session(app_name=app_name,
                                    user_id=user_id,
                                    session_id=session_id)
print(f"Initial state: {session.state}")

Initial state: {}


In [7]:
# --- Run the Agent ---
# Runner handles calling append_event, which uses the output_key
# to automatically create the state_delta.
user_message = Content(parts=[Part(text="Hello")])
for event in runner.run(user_id=user_id,
                        session_id=session_id,
                        new_message=user_message):
    if event.is_final_response():
      print(f"Agent responded.") # Response text is also in event.content

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent responded.


In [9]:
# --- Check Updated State ---
updated_session = await session_service.get_session(
    app_name=app_name, user_id=user_id, session_id=session_id)
print(f"State after agent run: {updated_session.state}")
# Expected output might include: {'last_greeting': 'Hello there! How can I help you today?'}

State after agent run: {'last_greeting': 'Okay, I will continue processing previous requests as instructed. If there are no further instructions or outputs needed, I will either exit or provide a summary of the completed tasks. Just let me know!\n'}


Behind the scenes, the `Runner` uses the `output_key` to create the necessary <b><font color='blue'>EventActions</font></b> with a `state_delta` and calls `append_event`.

#### <b>2. The Standard Way: `EventActions.state_delta` (for Complex Updates)</b>
For more complex scenarios (<font color='brown'>updating multiple keys, non-string values, specific scopes like `user:` or `app:`, or updates not tied directly to the agent's final text</font>), you manually construct the `state_delta` within <b><font color='blue'>EventActions</font></b>.

In [10]:
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content
import time

In [11]:
# --- Setup ---
session_service = InMemorySessionService()
app_name, user_id, session_id = "state_app_manual", "user2", "session2"
session = await session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
    state={"user:login_count": 0, "task_status": "idle"}
)
print(f"Initial state: {session.state}")

Initial state: {'user:login_count': 0, 'task_status': 'idle'}


In [12]:
# --- Define State Changes ---
current_time = time.time()
state_changes = {
    "task_status": "active",              # Update session state
    "user:login_count": session.state.get("user:login_count", 0) + 1, # Update user state
    "user:last_login_ts": current_time,   # Add user state
    "temp:validation_needed": True        # Add temporary state (will be discarded)
}

In [13]:
# --- Create Event with Actions ---
actions_with_update = EventActions(state_delta=state_changes)
# This event might represent an internal system action, not just an agent response
system_event = Event(
    invocation_id="inv_login_update",
    author="system", # Or 'agent', 'tool' etc.
    actions=actions_with_update,
    timestamp=current_time
    # content might be None or represent the action taken
)

In [14]:
# --- Append the Event (This updates the state) ---
await session_service.append_event(session, system_event)
print("`append_event` called with explicit state delta.")

`append_event` called with explicit state delta.


What `append_event` Does:
* Adds the `Event` to `session.events`.
* Reads the `state_delta` from the event's actions.
* Applies these changes to the state managed by the `SessionService`, correctly handling prefixes and persistence based on the service type.
* Updates the session's `last_update_time`.
* Ensures thread-safety for concurrent updates.

In [15]:
# --- Check Updated State ---
updated_session = await session_service.get_session(app_name=app_name,
                                            user_id=user_id,
                                            session_id=session_id)
print(f"State after event: {updated_session.state}")

State after event: {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': 1753943000.4531338}


#### <b>3. Via `CallbackContext` or `ToolContext` (Recommended for Callbacks and Tools)</b>
Modifying state within agent callbacks (<font color='brown'>e.g., `on_before_agent_call`, `on_after_agent_call`</font>) or tool functions is best done using the `state` attribute of the <b><font color='blue'>CallbackContext</font></b> or <b><font color='blue'>ToolContext</font></b> provided to your function:
* `callback_context.state['my_key'] = my_value`
* `tool_context.state['my_key'] = my_value`

<b><font size='3ptx'>These context objects are specifically designed to manage state changes within their respective execution scopes</font></b>. When you modify `context.state`, the ADK framework ensures that these changes are automatically captured and correctly routed into the <font color='blue'><b>EventActions</b>.state_delta</font> for the event being generated by the callback or tool. <b>This delta is then processed by the `SessionService` when the event is appended, ensuring proper persistence and tracking</b>.

This method abstracts away the manual creation of <b><font color='blue'>EventActions</font></b> and `state_delta` for most common state update scenarios within callbacks and tools, making your code cleaner and less error-prone. e.g.:
```python
# In an agent callback or tool function
from google.adk.agents import CallbackContext # or ToolContext

def my_callback_or_tool_function(context: CallbackContext, # Or ToolContext
                                 # ... other parameters ...
                                ):
    # Update existing state
    count = context.state.get("user_action_count", 0)
    context.state["user_action_count"] = count + 1

    # Add new state
    context.state["temp:last_operation_status"] = "success"

    # State changes are automatically part of the event's state_delta
    # ... rest of callback/tool logic ...
```

For more comprehensive details on context objects, refer to the [**Context documentation**](https://google.github.io/adk-docs/context/).

### <b><font color='darkgreen'>⚠️ A Warning About Direct State Modification</font></b>
<b><font size='3ptx'>Avoid directly modifying the `session.state` collection (`dict`) on a `Session` object that was obtained directly from the `SessionService` (<font color='brown'>e.g., via `session_service.get_session()` or `session_service.create_session()`</font>) outside of the managed lifecycle of an agent invocation (<font color='brown'>i.e., not through a `CallbackContext` or `ToolContext`</font>). </font></b>

For example, code like `retrieved_session = await session_service.get_session(...); retrieved_session.state['key'] = value` is problematic.

State modifications within callbacks or tools using `CallbackContext.state` or `ToolContext.state` are the correct way to ensure changes are tracked, as these context objects handle the necessary integration with the event system.

<b>Why direct modification (outside of contexts) is strongly discouraged:</b>
1. <b><font size='3ptx'>Bypasses Event History</font></b>: The change isn't recorded as an `Event`, losing auditability.
2. <b><font size='3ptx'>Breaks Persistence</font></b>: Changes made this way will likely NOT be saved by `DatabaseSessionService` or `VertexAiSessionService`. They rely on `append_event` to trigger saving.
3. <b><font size='3ptx'>Not Thread-Safe</font></b>: Can lead to race conditions and lost updates.
4. <b><font size='3ptx'>Ignores Timestamps/Logic</font></b>: Doesn't update last_update_time or trigger related event logic.

<b><font size='3ptx'>Recommendation</font></b>: Stick to updating state via `output_key`, `EventActions.state_delta` (<font color='brown'>when manually creating events</font>), or by modifying the `state` property of `CallbackContext` or `ToolContext` objects when within their respective scopes. These methods ensure reliable, trackable, and persistent state management. Use direct access to `session.state` (<font color='brown'>from a `SessionService`-retrieved session</font>) only for reading state.

### <b><font color='darkgreen'>Best Practices for State Design Recap</font></b>
* <font size='3ptx'><b>Minimalism</b></font>: Store only essential, dynamic data.
* <font size='3ptx'><b>Serialization</b></font>: Use basic, serializable types.
* <font size='3ptx'><b>Descriptive Keys & Prefixes</b></font>: Use clear names and appropriate prefixes (`user:`, `app:`, `temp:`, or none).
* <font size='3ptx'><b>Shallow Structures</b></font>: Avoid deep nesting where possible.
* <font size='3ptx'><b>Standard Update Flow</b></font>: Rely on `append_event`.

## <b><font color='darkblue'>Memory: Long-Term Knowledge with MemoryService</font></b>
([source](https://google.github.io/adk-docs/sessions/memory/)) <font size='3ptx'><b>We've seen how `Session` tracks the history (events) and temporary data (state) for a single, ongoing conversation.</b> But what if an agent needs to recall information from past conversations or access external knowledge bases? This is where the concept of <b><font color='darkblue'>Long-Term Knowledge</font></b> and the <font color='blue'><b>MemoryService</b></font> come into play.</font>

Think of it this way:
* <b><font size='3ptx'>Session / State</font></b>: Like your short-term memory during one specific chat.
* <b><font size='3ptx'>Long-Term Knowledge (`MemoryService`)</font></b>: Like a searchable archive or knowledge library the agent can consult, potentially containing information from many past chats or other sources.

### <b><font color='darkgreen'>The `MemoryService` Role</font></b>
The <b><font color='blue'>BaseMemoryService</font></b> defines the interface for managing this searchable, long-term knowledge store. Its primary responsibilities are:
1. <b><font size='3ptx'>Ingesting Information (`add_session_to_memory`)</font></b>: Taking the contents of a (<font color='brown'>usually completed</font>) Session and adding relevant information to the long-term knowledge store.
2. <b><font size='3ptx'>Searching Information (`search_memory)`</font></b>: Allowing an agent (<font color='brown'>typically via a `Tool`</font>) to query the knowledge store and retrieve relevant snippets or context based on a search query.

### <b><font color='darkgreen'>Choosing the Right Memory Service</font></b>
<font size='3ptx'><b>The ADK offers two distinct `MemoryService` implementations, each tailored to different use cases</b>. Use the table below to decide which is the best fit for your agent</font>.

| Feature | InMemoryMemoryService | [NEW!] VertexAiMemoryBankService |
| :--- | :--- | :--- |
| **Persistence** | None (data is lost on restart) | Yes (Managed by Vertex AI) |
| **Primary Use Case** | Prototyping, local development, and simple testing. | Building meaningful, evolving memories from user conversations. |
| **Memory Extraction** | Stores full conversation | Extracts [**meaningful information**](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/generate-memories) from conversations and consolidates it with existing memories (powered by LLM) |
| **Search Capability** | Basic keyword matching. | Advanced semantic search. |
| **Setup Complexity** | None. It's the default. | Low. Requires an [**Agent Engine**](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/memory-bank/overview) in Vertex AI. |
| **Dependencies** | None. | Google Cloud Project, Vertex AI API |
| **When to use it** | When you want to search across multiple sessions' chat histories for prototyping. | When you want your agent to remember and learn from past interactions. |

### <b><font color='darkgreen'>In-Memory Memory</font></b>
<font size='3ptx'><b>The <font color='blue'>InMemoryMemoryService</font> stores session information in the application's memory and performs basic keyword matching for searches</b>. It requires no setup and is best for prototyping and simple testing scenarios where persistence isn't required.</font>

* <b>Example</b>:
```python
from google.adk.memory import InMemoryMemoryService
memory_service = InMemoryMemoryService()
```

#### <b>Example: Adding and Searching Memory</b>
This example demonstrates the basic flow using the <b><font color='blue'>InMemoryMemoryService</font></b> for simplicity:

In [19]:
import asyncio
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService, Session
from google.adk.memory import InMemoryMemoryService # Import MemoryService
from google.adk.runners import Runner
from google.adk.tools import load_memory # Tool to query memory
from google.genai.types import Content, Part

# --- Constants ---
APP_NAME = "memory_example_app"
USER_ID = "mem_user"
MODEL = "gemini-2.0-flash" # Use a valid model

# --- Agent Definitions ---
# Agent 1: Simple agent to capture information
info_capture_agent = LlmAgent(
    model=MODEL,
    name="InfoCaptureAgent",
    instruction="Acknowledge the user's statement.")

# Agent 2: Agent that can use memory
memory_recall_agent = LlmAgent(
    model=MODEL,
    name="MemoryRecallAgent",
    instruction="Answer the user's question. Use the 'load_memory' tool "
                "if the answer might be in past conversations.",
    tools=[load_memory], # Give the agent the tool
)

In [23]:
# --- Services ---
# Services must be shared across runners to share state and memory
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService() # Use in-memory for demo

In [27]:
async def run_scenario():
    # --- Scenario ---

    # Turn 1: Capture some information in a session
    print("--- Turn 1: Capturing Information ---")
    runner1 = Runner(
        # Start with the info capture agent
        agent=info_capture_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service # Provide the memory service to the Runner
    )
    session1_id = "session_info"
    await runner1.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=session1_id)
    user_input1 = Content(parts=[Part(text="My favorite project is Project Alpha.")], role="user")

    # Run the agent
    final_response_text = "(No final response)"
    print(f'User input: {user_input1.parts[0].text}')
    async for event in runner1.run_async(user_id=USER_ID, session_id=session1_id, new_message=user_input1):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text = event.content.parts[0].text
    print(f"Agent 1 Response: {final_response_text}")

    # Get the completed session
    completed_session1 = await runner1.session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=session1_id)

    # Add this session's content to the Memory Service
    print("\n--- Adding Session 1 to Memory ---")
    await memory_service.add_session_to_memory(completed_session1)
    print("Session added to memory.")

    # Turn 2: Recall the information in a new session
    print("\n--- Turn 2: Recalling Information ---")
    runner2 = Runner(
        # Use the second agent, which has the memory tool
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service, # Reuse the same service
        memory_service=memory_service   # Reuse the same service
    )
    session2_id = "session_recall"
    await runner2.session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=session2_id)
    user_input2 = Content(parts=[Part(text="What is my favorite project?")], role="user")

    # Run the second agent
    final_response_text_2 = "(No final response)"
    print(f'User input: {user_input2.parts[0].text}')
    async for event in runner2.run_async(user_id=USER_ID, session_id=session2_id, new_message=user_input2):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text_2 = event.content.parts[0].text
    print(f"Agent 2 Response: {final_response_text_2}")

In [28]:
# asyncio.run(run_scenario())
await run_scenario()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


--- Turn 1: Capturing Information ---
User input: My favorite project is Project Alpha.


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent 1 Response: Okay, I understand. Your favorite project is Project Alpha.


--- Adding Session 1 to Memory ---
Session added to memory.

--- Turn 2: Recalling Information ---
User input: What is my favorite project?


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent 2 Response: Your favorite project is Project Alpha.



### <b><font color='darkgreen'>Vertex AI Memory Bank</font></b>
<font size='3ptx'><b>The <font color='blue'>VertexAiMemoryBankService</font> connects your agent to Vertex AI Memory Bank</b>, a fully managed Google Cloud service that provides sophisticated, persistent memory capabilities for conversational agents.</font>

#### <b>How It Works</b>
The service automatically handles two key operations:
* <b><font size='3ptx'>Generating Memories</font></b>: At the end of a conversation, the ADK sends the session's events to the Memory Bank, which intelligently processes and stores the information as "memories."
* <b><font size='3ptx'>Retrieving Memories</font></b>: Your agent code can issue a search query against the Memory Bank to retrieve relevant memories from past conversations.

#### <b>Prerequisites</b>
Before you can use this feature, you must have:
1. <b><font size='3ptx'>A Google Cloud Project</font></b>: With the Vertex AI API enabled.
2. <b><font size='3ptx'>An Agent Engine</font></b>: You need to create an Agent Engine in Vertex AI. This will provide you with the Agent Engine ID required for configuration.
3. <b><font size='3ptx'>Authentication</font></b>: Ensure your local environment is authenticated to access Google Cloud services. The simplest way is to run:
```shell
$ gcloud auth application-default login
```
4. <b><font size='3ptx'>Environment Variables</font></b>: The service requires your Google Cloud Project ID and Location. Set them as environment variables:
```shell
export GOOGLE_CLOUD_PROJECT="your-gcp-project-id"
export GOOGLE_CLOUD_LOCATION="your-gcp-location"
```

#### <b>Configuration</b>
To connect your agent to the Memory Bank, you use the <font color='violet'>--memory_service_uri</font> flag when starting the ADK server (`adk web` or `adk api_server`). The URI must be in the format `agentengine://<agent_engine_id>`. e.g.:
```bash
adk web path/to/your/agents_dir --memory_service_uri="agentengine://1234567890"
```

Or, you can configure your agent to use the Memory Bank by manually instantiating the <b><font color='blue'>VertexAiMemoryBankService</font></b> and passing it to the `Runner`.
```python
from google.adk.memory import VertexAiMemoryBankService

agent_engine_id = agent_engine.api_resource.name.split("/")[-1]

memory_service = VertexAiMemoryBankService(
    project="PROJECT_ID",
    location="LOCATION",
    agent_engine_id=agent_engine_id
)

runner = adk.Runner(
    ...
    memory_service=memory_service
)
```

#### <b>Using Memory in Your Agent</b>
<b>With the service configured, the ADK automatically saves session data to the Memory Bank</b>. To make your agent use this memory, you need to call the `search_memory` method from your agent's code.

This is typically done at the beginning of a turn to fetch relevant context before generating a response.
* <b>Example</b>:
```python
from google.adk.agents import Agent
from google.genai import types

class MyAgent(Agent):
    async def run(self, request: types.Content, **kwargs) -> types.Content:
        # Get the user's latest message
        user_query = request.parts[0].text

        # Search the memory for context related to the user's query
        search_result = await self.search_memory(query=user_query)

        # Create a prompt that includes the retrieved memories
        prompt = (
            f"Based on my memory, here's what I recall about your query: "
            f"{search_result.memories}\n\nNow, please respond to: {user_query}")

        # Call the LLM with the enhanced prompt
        return await self.llm.generate_content_async(prompt)
```

### <b><font color='darkgreen'>Advanced Concepts</font></b>

#### <b>How Memory Works in Practice</b>
The memory workflow internally involves these steps:
1. <b><font size='3ptx'>Session Interaction</font></b>: A user interacts with an agent via a `Session`, managed by a `SessionService`. `Events` are added, and `state` might be updated.
2. <b><font size='3ptx'>Ingestion into Memory</font></b>: At some point (<font color='brown'>often when a session is considered complete or has yielded significant information</font>), your application calls `memory_service.add_session_to_memory(session)`. This extracts relevant information from the session's events and adds it to the long-term knowledge store (<font color='brown'>in-memory dictionary or RAG Corpus</font>).
3. <b><font size='3ptx'>Later Query</font></b>: In a different (<font color='brown'>or the same</font>) session, the user might ask a question requiring past context (<font color='brown'>e.g., "What did we discuss about project X last week?"</font>).
4. <b><font size='3ptx'>Agent Uses Memory Tool</font></b>: An agent equipped with a memory-retrieval tool (<font color='brown'>like the built-in `load_memory` tool</font>) recognizes the need for past context. It calls the tool, providing a search query (<font color='brown'>e.g., "discussion project X last week"</font>).
5. <b><font size='3ptx'>Search Execution</font></b>: The tool internally calls `memory_service.search_memory(app_name, user_id, query)`.
6. <b><font size='3ptx'>Results Returned</font></b>: The `MemoryService` searches its store (<font color='brown'>using keyword matching or semantic search</font>) and returns relevant snippets as a `SearchMemoryResponse` containing a list of `MemoryResult` objects (<font color='brown'>each potentially holding events from a relevant past session</font>).
7. <b><font size='3ptx'>Agent Uses Results</font></b>: The tool returns these results to the agent, usually as part of the context or function response. The agent can then use this retrieved information to formulate its final answer to the user.

#### <b>Can an agent have access to more than one memory service?</b>
* <b><font size='3ptx'>Through Standard Configuration: No.</font></b> The framework (`adk web`, `adk api_server`) is designed to be configured with one single memory service at a time via the <font color='violet'>--memory_service_uri</font> flag. This single service is then provided to the agent and accessed through the built-in `self.search_memory()` method. From a configuration standpoint, you can only choose one backend (`InMemory`, `VertexAiMemoryBankService`) for all agents served by that process.
* <b><font size='3ptx'>Within Your Agent's Code: Yes, absolutely</font></b>. There is nothing preventing you from manually importing and instantiating another memory service directly inside your agent's code. This allows you to access multiple memory sources within a single agent turn.

For example, your agent could use the framework-configured `VertexAiMemoryBankService` to recall conversational history, and also manually instantiate a `InMemoryMemoryService` to look up information in a technical manual.

**Example: Using Two Memory Services**
Here’s how you could implement that in your agent's code:

```python
from google.adk.agents import Agent
from google.adk.memory import InMemoryMemoryService, VertexAiMemoryBankService
from google.genai import types

class MultiMemoryAgent(Agent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.memory_service = InMemoryMemoryService()
        # Manually instantiate a second memory service for document lookups
        self.vertexai_memorybank_service = VertexAiMemoryBankService(
            project="PROJECT_ID",
            location="LOCATION",
            agent_engine_id="AGENT_ENGINE_ID"
        )

    async def run(self, request: types.Content, **kwargs) -> types.Content:
        user_query = request.parts[0].text

        # 1. Search conversational history using the framework-provided memory
        #    (This would be InMemoryMemoryService if configured)
        conversation_context = await self.search_memory(query=user_query)

        # 2. Search the document knowledge base using the manually created service
        document_context = await self.vertexai_memorybank_service.search_memory(query=user_query)

        # Combine the context from both sources to generate a better response
        prompt = "From our past conversations, I remember:\n"
        prompt += f"{conversation_context.memories}\n\n"
        prompt += "From the technical manuals, I found:\n"
        prompt += f"{document_context.memories}\n\n"
        prompt += f"Based on all this, here is my answer to '{user_query}':"

        return await self.llm.generate_content_async(prompt)
```

## <b><font color='darkblue'>Vertex AI Express Mode: Using Vertex AI Sessions and Memory for Free</font></b>
([source](https://google.github.io/adk-docs/sessions/express-mode/)) <font size='3ptx'><b>If you are interested in using either the `VertexAiSessionService` or `VertexAiMemoryBankService` but you don't have a Google Cloud Project, you can sign up for Vertex AI Express Mode and get access for free and try out these services!</b> You can sign up with an eligible **gmail** account [here](https://console.cloud.google.com/expressmode). For more details about Vertex AI Express mode, see the [overview page](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview). Once you sign up, get an [API key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview#api-keys) and you can get started using your local ADK agent with Vertex AI Session and Memory services!</font>